# mOWL: Python library for machine learning with ontologies

Ontologies are being used across different machine learning models
since they can provide structured and axiomatic background knowldege regarding a particular domain of interest.


<img src="figs/mowl_intro/ont_repr.jpg" width="600" alt="Ontology with axioms and annotations." title="Ontology with axioms and annotations."/>

In mOWL, we have implemented a set of tools and functionalities to ease the integration of methods that use ontologies. The features that mOWL contains are:

* Ontology Management
* Ontology Transformation
* Embedding Generation
* Embedding Post-processing

## Ontology Management

This includes ontology  normalization and reasoning. To achieve this, we interface with the OWL API.

<img src="figs/mowl_intro/reasoning.jpg" width="400" alt="Ontology reasoning." title="Ontology reasoning."/>

## Ontology Transformation
This includes methods for extracting ontology information into graphs, text corpora or process the axioms directly.

<img src="figs/mowl_intro/transformation.jpg" width="400" alt="Ontology transformation." title="Ontology transformation."/>

## Embedding Generation

Each ontology transformation result is usable to generate vector representations (embeddings) of entities. For graph projections we can use random walk methods (implemented in mOWL) or interface with the knowledge graph embedding library PyKEEN. Transformation into text allows to use natural language processing methods.

<img src="figs/mowl_intro/embeddings.jpg" width="600" alt="Embeddings generation." title="Embeddings generation."/>

## Embedding post-processing

This includes methods to use the embeddings previously generated. We can directly evaluate them using a set of metrics or use them to train a machine learning model.

# Getting started

To get started, you can install mOWL using pip:

In [ ]:
!pip install mowl-borg

mOWL interfaces the OWL API. For this, we need to interface with the Java Virtual Macine (JVM):

In [2]:
import mowl
mowl.init_jvm("10g")